# Limitations with Large Language Models

Large Language Models are objectively great. They are flexible, surprisingly cunning, and have a considerable amount of knowledge by themselves. They do come short in some cases, especially when it comes to adapting to new contextual information. Let's say you're trying to build an LLM that answers all the questions you may have about BeCode rules. What does ChatGPT know about BeCode rules, was there any of it in its training data? Probably not much.

## How could we have a LLM answer BeCode questions?

An LLM has a very long context window, close to 1 million words for ChatGPT, that means close to two books from Game of Thrones can be given to it and it would still be able to answer. We could give it all of BeCode rules as a text in the prompt and have it answer questions based on them. But it still comes with many caveats, mostly that giving a lot of content to an LLM is quite costly in resources and money.

Wouldn't it be better if we could just give it the parts of the document useful in the answer to help it on the prompt at hand? The LLM doesn't need to be told about the way moodle works in order to explain when the holidays of the bootcamp happen. The document with Becode Rules is given in `data/becode_rules.txt`.

Make it so that the LLM can answer the following prompt by giving it the paragraph from becode_rules that will allow it to answer the prompt, insert this in the code snippet underneath:

In [ ]:
# Import the Python SDK
from google import genai

API_KEY="GEMINI API KEY HERE"

if API_KEY=="GEMINI API KEY HERE":
    raise Exception('Your API key has to be put instead of "GEMINI API KEY HERE"')

client = genai.Client(api_key=API_KEY)#Telling google what your API key is

question = 'I am sick, I sent an email to Antoine and Cindy, what else should I do?'
context = '''PASTE INFORMATION IN THE DOCUMENT ABOUT BEING SICK'''
prompt = f'Use the following snippet:\n {context}\n\n To answer this question: {question}'
print("Prompt:\n",prompt)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
)
print("\n\nAnswer:\n", response.text)

## Word2Vec makes a comeback

You just had to give the answer to ChatGPT to have it tell you what to do. Not very handy, might as well go search in the document yourself. But what if there was a way to make a program perform that search automatically?

In the previous notebooks, you may have read that we used to turn some words into vectors to encode meaning about them, and that words with similar meanings had similar vectors. Well what if you could do this instead with many words? What if you could do it with paragraphs? Wouldn't that be great.

Well as it turns out, you can, you can make [embeddings for paragraphs](https://ai.google.dev/gemini-api/docs/embeddings). You can do that with an entire document, and then use the paragraph who's vectors are similar to your prompt to augment it!

In [ ]:
prompt_embed = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents="I am sick, I sent an email to Antoine and Cindy, what else should I do?")

print(prompt_embed.embeddings)

In [ ]:
doc_content=""
with open('data/becode_rules.txt','r',encoding="utf-8") as f:
    doc_content=f.read()

chunk_size=500
chunks=[]
nb_full_chunks=len(doc_content)//chunk_size

for i in range(nb_full_chunks):
    chunks.append(doc_content[i*chunk_size:(i+1)*chunk_size])
last_chunk_start=nb_full_chunks*chunk_size
chunks.append(doc_content[last_chunk_start:last_chunk_start+len(doc_content)%chunk_size])

print(chunks)

Run this:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Prompt embedding
prompt_vector = np.array(prompt_embed.embeddings[0].values).reshape(1, -1)

# Chunk 3 and 4 embeddings
chunk_3_vector = np.array(
    client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=chunks[3]
    ).embeddings[0].values
).reshape(1, -1)

chunk_4_vector = np.array(
    client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=chunks[4]
    ).embeddings[0].values
).reshape(1, -1)

# Print the text
print("Chunk 3:")
print(chunks[3])

print("\nChunk 4:")
print(chunks[4])

# Compute cosine similarities
sim_3 = cosine_similarity(prompt_vector, chunk_3_vector)[0][0]
sim_4 = cosine_similarity(prompt_vector, chunk_4_vector)[0][0]

# Display results
print(f"\nSimilarity between the prompt and Chunk 3: {sim_3:.4f}")
print(f"Similarity between the prompt and Chunk 4: {sim_4:.4f}")

print("Is this giving you any ideas????")

Take time to understand the example given above. Then try to make a system which can answer any question by selecting the best chunks from the content to answer the prompt. You may explore different chunk sizes, some overlap between chunks and a criterion for minimum required similarity. Give it a try!

In [ ]:
#Your code here

## Where is this going

I hope you'll quickly realise how powerful this approach can be when augmenting the prompts of LLMS. You just need to find the segments with the highest similarity and feed them in the prompt. This is the beauty of Retrieval Augmented Generation (RAG). There are many things to explore from here and I invite you to go look for these topics in whichever way you prefer. Here is an example list of ideas:

- You can look into different embedding models for sentences and paragraphs. This is the current overall [leaderboard](https://huggingface.co/spaces/mteb/leaderboard)
- You can look into ways google has of using sentence embeddings which may be a little less clunky than what we did above
- You can have a peek into ways of augmenting RAG with a classic keywords search
- You can try to replicate this example with something you care about

Have fun with it!